# 1D Nozzle simulator

Author: Shaowu Pan


# Setup the area profile


In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import scienceplots
import scipy
plt.style.use(['science','ieee'])

class Nozzle(object):
    def __init__(self, Afunc, xmin, xmax, gamma, R) -> None:
        self.A = Afunc
        self.xmin = xmin 
        self.xmax = xmax 
        self.x = np.linspace(self.xmin,self.xmax,1000,endpoint=True)
        self.area_array = self.A(self.x)
        self.area_exit = self.A(self.xmax)
        self.x_throat = scipy.optimize.fmin(self.A, x0 = 0.5*(xmin+xmax))[0]
        self.area_throat = self.A(self.x_throat)
        self.area_array_before_throat = self.area_array[self.x<=self.x_throat]
        self.area_array_after_throat = self.area_array[self.x>=self.x_throat]
        self.g = gamma
        self.R = R

        # compute critical pressures 1-2-3
        # Critical case pressure ratio(s) for Case 1 
        ratio = self.get_exit_area_over_throat()
        m_crit_1 = self.solve_mach_number_from_area_ratio(ratio=ratio, gamma=gamma, is_subsonic=True)
        p0_p = (1+(gamma-1)/2*m_crit_1**2)**(gamma/(gamma-1))
        self.crit_p_ratio_1 = 1/p0_p        

        # Critical case pressure ratio(s) for Case 2 - normal shock
        m_crit_2 = self.solve_mach_number_from_area_ratio(ratio=ratio, gamma=gamma, is_subsonic=False)
        p0_pe = (1+(gamma-1)/2*m_crit_2**2)**(gamma/(gamma-1))
        # normal shock
        p_ns_ratio = 1 + 2*gamma/(gamma+1)*(m_crit_2**2-1)
        self.crit_p_ratio_2 = p_ns_ratio/p0_pe

        # Critical case pressure ratio(s) for Case 3 - shockfree
        self.crit_p_ratio_3 = 1/p0_pe

        print(f"critical pressure rato (p_b/p_0) - chock =          {self.crit_p_ratio_1:.5f}")
        print(f"critical pressure rato (p_b/p_0) - normal shock =   {self.crit_p_ratio_2:.5f}")
        print(f"critical pressure rato (p_b/p_0) - shock free =     {self.crit_p_ratio_3:.5f}")

    def plot_area_profile(self):
        plt.figure(figsize=(4,4))
        plt.plot(self.x,self.area_array)
        plt.xlabel('x',size=15)
        plt.ylabel('A(x)',size=15)
        plt.xlim([0, 1])
        # plt.ylim([0, 0.6])
        plt.grid('on')
        plt.title('area profile of a Laval nozzle') 

    @classmethod
    def area_mach_relation(cls, m, gamma):
        A_over_A_throat = np.sqrt((2/(gamma+1)*(1+(gamma-1)/2*m**2))**((gamma+1)/(gamma-1))/m**2)
        return A_over_A_throat
    
    # entropy rise across a normal shock as a function of M1
    @classmethod
    def entropy_jump_normal_shock(cls, M1, gamma, R):
        cp = gamma * R / (gamma - 1.0)
        pressure_ratio = 1 + (2 * gamma / (gamma + 1.0)) * (M1**2 - 1.0)
        temp_ratio = pressure_ratio * (2 + (gamma - 1.0) * M1**2) / ((gamma + 1.0) * M1**2)
        return cp * np.log(temp_ratio) - R * np.log(pressure_ratio)


    def plot_flow_profile(self, pb_p0_ratio):

        if pb_p0_ratio > self.crit_p_ratio_1:
            print(f"===\npb/p0 = {pb_p0_ratio:.3f} --> Entirely subsonic \n===")

            p0_pb = 1.0/pb_p0_ratio
            m_exit = np.sqrt((p0_pb**((self.g-1)/(self.g)) - 1)*2/(self.g-1))
            Ae_over_A_star = self.area_mach_relation(m_exit, self.g)
            A_over_A_star = self.area_array / self.area_exit * Ae_over_A_star

            # inverse solving M(x) from area ratio 
            M_array = np.zeros_like(self.x)
            p_array = np.zeros_like(self.x)
            for i, ratio in enumerate(A_over_A_star):
                # print("A over A* = ",ratio, "at x = ", self.x[i])
                M_array[i] = self.solve_mach_number_from_area_ratio(ratio, self.g, is_subsonic=True)
                p_array[i] = 1.0/p0_pb**((self.g-1)/(self.g)) * (1 + (self.g-1)/2*M_array[i]**2)**(-self.g/(self.g-1))

        elif pb_p0_ratio > self.crit_p_ratio_2:
            print(f"===\npb/p0 = {pb_p0_ratio:.3f} --> Normal shock inside the expansion \n===")
            p0_pb = 1.0/pb_p0_ratio
            
            # search for x_shock >= x_throat, but x_shock < x_max
            # we will try to define a function of x_shock, and try to match the pb_p0_ratio with the normal shock condition

            
            def M_and_p_given_x_shock(x_shock):
                last_index_before_shock = np.where(self.x < x_shock)[0][-1]
                M_array = np.zeros_like(self.x)
                p_array = np.zeros_like(self.x)
                A_over_A_star = self.area_array / self.area_throat 

                for i, ratio in enumerate(A_over_A_star):
                    if self.x[i] < self.x_throat:
                        M_array[i] = self.solve_mach_number_from_area_ratio(ratio, self.g, is_subsonic=True)
                        p_array[i] = 1.0/p0_pb**((self.g-1)/(self.g)) * (1 + (self.g-1)/2*M_array[i]**2)**(-self.g/(self.g-1))
                    elif self.x[i] < x_shock:
                        M_array[i] = self.solve_mach_number_from_area_ratio(ratio, self.g, is_subsonic=False)
                        p_array[i] = 1.0/p0_pb**((self.g-1)/(self.g)) * (1 + (self.g-1)/2*M_array[i]**2)**(-self.g/(self.g-1))
                    else:                     
                        M1 = M_array[last_index_before_shock]
                        # after the shock 
                        # first calculate total pressure loss
                        delta_s = self.entropy_jump_normal_shock(M1,self.g, R=self.R)
                        p0_new_p0 = np.exp(-delta_s/self.R)
                        # p0_new_p_exit = p0_new_p0 * p0_pb
                        M2 = np.sqrt((1+(self.g-1)/2*M1**2)/(self.g*M1**2-(self.g-1)/2))

                        # A / A_shock
                        A_over_A_shock = ratio*self.area_throat / self.get_area(self.x[last_index_before_shock+1])

                        # A_shock / A_star
                        A_shock_over_A_star = self.area_mach_relation(M2, self.g)

                        # A / A_star
                        A_over_A_star_tmp = A_over_A_shock * A_shock_over_A_star

                        # inverse modeling to find M 
                        M_array[i] = self.solve_mach_number_from_area_ratio(A_over_A_star_tmp, self.g, is_subsonic=True)
                        p_array[i] = p0_new_p0 * (1 + (self.g-1)/2*M_array[i]**2)**(-self.g/(self.g-1))
                return M_array, p_array

            # find x_shock 
            predicted_exit_pressure = lambda x_shock: M_and_p_given_x_shock(x_shock)[1][-1] - pb_p0_ratio
            x_shock = scipy.optimize.root_scalar(predicted_exit_pressure, 
                                                 bracket=[self.x_throat, self.xmax], 
                                                 method='brentq', maxiter=1000,xtol=1e-7,rtol=1e-7).root
            M_array, p_array = M_and_p_given_x_shock(x_shock)

        elif pb_p0_ratio > self.crit_p_ratio_3:
            pass
        else:
            pass 
        # draw m vs x
        # draw p vs x
        plt.figure(figsize=(4,2))
        plt.plot(self.x,self.area_array,label='$A(x)$')
        plt.xlabel('$x$',size=15)
        # plt.ylabel('A(x)',size=25)
        plt.plot(self.x,M_array,label='$M(x)$')
        plt.plot(self.x,p_array,label=r'$p/p_0(x)$')
        plt.xlim([0,1])
        # plt.ylim([0,2])
        plt.grid('on')
        plt.legend(loc='best')

        pass 

    def get_area(self, x):
        return self.A(x)

    def get_area_over_throat_at_x(self, x):
        return self.A(x)/self.area_throat

    def get_exit_area_over_throat(self):
        return self.area_exit/self.area_throat

    def get_area_over_throat(self, A):
        return A/self.area_throat

    def solve_mach_number_from_area_ratio(self, ratio, gamma, is_subsonic=True):
        eq = lambda m: m**2*ratio**2 - (2/(gamma+1)*(1+(gamma-1)/2*m**2))**((gamma+1)/(gamma-1)) 
        if is_subsonic:
            sol = scipy.optimize.root_scalar(eq, bracket=[0, 1], method='brentq', maxiter=1000,xtol=1e-7,rtol=1e-7)
        else:
            sol = scipy.optimize.root_scalar(eq, bracket=[1, 10], method='brentq', maxiter=1000,xtol=1e-7,rtol=1e-7)
        return sol.root

    @property
    def area_exit(self):
        return self._area_exit

    @area_exit.setter
    def area_exit(self,value):
        self._area_exit = value

    @property
    def area_throat(self):
        return self._area_throat
        
    @area_throat.setter
    def area_throat(self,value):
        self._area_throat = value







In [ ]:
# geometry of the nozzle 

A = lambda x: 3*(x-0.6)**2+0.25


# Setup the inlet and outlet boundary conditions


In [ ]:
# gamma
g = 1.4
R = 287

# inflow condition
T_0 = 273
p_0 = 101325


In [ ]:
# 1: entirely subsonic 
p_ratio = 0.972
p_b = p_ratio*p_0

nozzle = Nozzle(A, xmin=0, xmax=1, gamma=g, R=R)
nozzle.plot_flow_profile(p_ratio)

In [ ]:
# 2: normal shock inside the expansion
p_ratio = 0.8
p_b = p_ratio*p_0

nozzle = Nozzle(A, xmin=0, xmax=1, gamma=g,R=R)
nozzle.plot_flow_profile(p_ratio)


